In [8]:

import pandas as pd
import json
from dotenv import load_dotenv
import os
import re
load_dotenv()

True

In [ ]:
sheet_name = "김희영"
book_name = "서초구청_업무메뉴얼"
page_number = 1

In [10]:
import gspread

json_file_path = "credentials.json"
gc = gspread.service_account(json_file_path)
spreadsheet_url = os.getenv("SPREADSHEET_URL")
doc = gc.open_by_url(spreadsheet_url)

worksheet = doc.worksheet(sheet_name)
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])  # 첫 번째 행을 컬럼 이름으로 사용
df["page_number"] = pd.to_numeric(df["page_number"], errors="coerce")
df.replace("", pd.NA, inplace=True)
df = df[(df['book_name'] == book_name) & (df["page_number"] == int(page_number))] # 책 이름 + 페이지 지정
df

,book_name,page_number,1,2,3,4,5,6,7,8,9,
2243,서초구청_업무메뉴얼,1.0,container,top2down,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2244,서초구청_업무메뉴얼,1.0,<NA>,container,left2right,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2245,서초구청_업무메뉴얼,1.0,<NA>,<NA>,textbox,top2down,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2246,서초구청_업무메뉴얼,1.0,<NA>,<NA>,<NA>,text,응답소 현장민원 관리,<NA>,<NA>,<NA>,<NA>,<NA>
2247,서초구청_업무메뉴얼,1.0,<NA>,<NA>,icon,https://github.com/hykim02/arasoft-dataset/blo...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2248,서초구청_업무메뉴얼,1.0,<NA>,<NA>,icon,https://github.com/hykim02/arasoft-dataset/blo...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2249,서초구청_업무메뉴얼,1.0,<NA>,textbox,top2down,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2250,서초구청_업무메뉴얼,1.0,<NA>,<NA>,text,"담당부서 l 감사담당관 l 담당팀 l 순찰팀 l 관련부서 l 전부서, 전동",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2251,서초구청_업무메뉴얼,1.0,<NA>,icon,https://github.com/hykim02/arasoft-dataset/blo...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2252,서초구청_업무메뉴얼,1.0,<NA>,layerlist_arccodion,top2down,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
class LayoutParser:
    def __init__(self):
        self.elements = {}
        self.element_counter = {}
    
    def remove_special_chars(self, text):
        # 특수문자 제거
        text = re.sub(r'[<>]', '', text)  # <와 > 제거
        text = re.sub(r'&[a-zA-Z]+;', '', text)  # HTML 엔티티 제거 (예: &amp;)
        return text
    
    def generate_element_id(self, element_type):
        if element_type not in self.element_counter:
            self.element_counter[element_type] = 1
        else:
            self.element_counter[element_type] += 1
        return f"{element_type}{self.element_counter[element_type]:03d}"

    def create_layout_tree(self, df):
        ELEMENTS = ['text', 'image', 'icon']
        CONTAINERS = {
            'container': ['top2down', 'left2right'],
            'textbox': ['top2down', 'left2right'],
            'layerlist_tab': ['top', 'down'],
            'layerlist_arccodion': ['top2down', 'left2right'],
            'layerlist_slider': ['top2down', 'left2right'],
            'nac_title': ['top2down', 'left2right'],
            'nac_item': ['top2down', 'left2right'],
        }
        
        def create_container_node(symbol_name, direction):
            return {
                "symbol_name": symbol_name,
                "direction": direction,
                "children": []
            }
        
        root = create_container_node("root", "top2down")
        stack = [(root, -1)]  # (node, depth) pairs
        
        for row in df.itertuples():
            row_data = list(row)[3:]
            
            # Find depth
            depth = 0
            for val in row_data:
                if pd.isna(val):
                    depth += 1
                else:
                    break
            
            symbol_name = row_data[depth]
            
            # Find content
            content = None
            for i in range(depth + 1, len(row_data)):
                if not pd.isna(row_data[i]) and row_data[i] not in CONTAINERS and \
                   row_data[i] not in ['top2down', 'left2right', 'top', 'down']:
                    if isinstance(row_data[i], str) and not row_data[i] in ELEMENTS:
                        content = row_data[i]
                        break
            
            # Create node
            if symbol_name in CONTAINERS:
                direction = row_data[depth + 1] if len(row_data) > depth + 1 and \
                          not pd.isna(row_data[depth + 1]) else "top2down"
                new_node = create_container_node(symbol_name, direction)
            elif symbol_name in ELEMENTS:
                # Generate unique ID for element
                element_id = self.generate_element_id(symbol_name)
                # Store element in elements dictionary
                self.elements[element_id] = {
                    "type": symbol_name,
                    "content": self.remove_special_chars(content)
                }
                # Use element_id as reference in tree
                new_node = element_id
            else:
                continue
            
            # Find appropriate parent and add node
            while stack and stack[-1][1] >= depth:
                stack.pop()
            
            if stack:
                if isinstance(new_node, dict):  # If container
                    stack[-1][0]["children"].append(new_node)
                    stack.append((new_node, depth))
                else:  # If element (string ID)
                    stack[-1][0]["children"].append(new_node)
        
        return {
            "elements": self.elements,
            "containers": root["children"][0] if root["children"] else {}
        }

In [16]:
parser = LayoutParser()
result = parser.create_layout_tree(df)

with open("output/manuscript.json", "w", encoding="utf-8") as json_file:
    json.dump(result['elements'], json_file, ensure_ascii=False, indent=4)
with open("output/symbolic_tree.json", "w", encoding="utf-8") as json_file:
    json.dump(result['containers'], json_file, ensure_ascii=False, indent=4)

응답소 현장민원 관리
https://github.com/hykim02/arasoft-dataset/blob/main/datasets/%EC%84%9C%EC%B4%88%EA%B5%AC%EC%B2%AD_%EC%97%85%EB%AC%B4%EB%A9%94%EB%89%B4%EC%96%BC(Page1)/image/img.png?raw=true
https://github.com/hykim02/arasoft-dataset/blob/main/datasets/%EC%84%9C%EC%B4%88%EA%B5%AC%EC%B2%AD_%EC%97%85%EB%AC%B4%EB%A9%94%EB%89%B4%EC%96%BC(Page1)/image/img2.png?raw=true
담당부서 l 감사담당관    l 담당팀 l 순찰팀    l 관련부서 l 전부서, 전동
https://github.com/hykim02/arasoft-dataset/blob/main/datasets/%EC%84%9C%EC%B4%88%EA%B5%AC%EC%B2%AD_%EC%97%85%EB%AC%B4%EB%A9%94%EB%89%B4%EC%96%BC(Page1)/image/tip.png?raw=true
업무개요
- 응답소 현장민원 처리부서 지정 및 처리현황 분석
업무 프로세스
<<응답소 현장민원>>
https://github.com/hykim02/arasoft-dataset/blob/main/datasets/%EC%84%9C%EC%B4%88%EA%B5%AC%EC%B2%AD_%EC%97%85%EB%AC%B4%EB%A9%94%EB%89%B4%EC%96%BC(Page1)/image/step1.png?raw=true
응답소 현장민원 접수 확인
-  응답소(120)에서 서초구 업무담당자에게 배분한 현장민원 확인(새올을 통해 응답소 프로그램 접속, 수시 확인) ※ 응답소 기관관리자 권한 필요(권한 요청은 서울시 민원담당관으로 공문 제출)
https://github.com/hykim02/arasoft-dataset/blob/main/datas